# Exact solution

Ethier C.R., Steinman D.A., Exact fully 3d Navier-Stokes solutions for banchmarking

Reference implementation with Taylor-Hood elements of degree 4/3

In [1]:
from dolfin import *
import numpy as np
import mshr as mshr

# Should stay 1 here (as we used normalized equation)
mu = Constant(1.)
muv = mu.values()[0]


baseR = 1.
topR = 1.5
h = 2.

outerCyl = mshr.Cylinder(Point([0.,0.,0.]),Point([0.,0.,h]),baseR,topR)
mesh=mshr.generate_mesh(outerCyl,15)

In [2]:
# We mark physical boundary and setup the space of finite elements 
mark = {"generic": 0,
        "wall": 1}

class Bondary_w(SubDomain):
    def inside(self, x, on_boundary):
        return on_boundary 

wall = Bondary_w()

#File("bord.pvd") << subdomains

degree = 4
P2 = VectorElement("P", mesh.ufl_cell(), degree)
P1 = FiniteElement("P", mesh.ufl_cell(), degree-1)
TH = MixedElement([P2, P1])


In [3]:


subdomains = MeshFunction("size_t", mesh,  mesh.topology().dim()-1)
subdomains.set_all(mark["generic"])
wall.mark(subdomains, mark["wall"])

W = FunctionSpace(mesh, TH)
bcu_wall = DirichletBC(W.sub(0), Constant((0.,0.,0.)), subdomains, mark["wall"])
dbcs = [bcu_wall]

Calling FFC just-in-time (JIT) compiler, this may take some time.


In [4]:
source = Expression('x[2]*x[2]*x[2]*x[2]*x[2]*x[2]*x[2]',degree=7)
w_old = Function(W)
(u1_o, u2_o) = split(w_old)
scale = assemble((u2_o-source)*dx)

Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.


In [5]:
# We define physical parameters

nbitt = 10

#T = 8.0
#timestep = Constant(T/nbitt)
#print(timestep.values()[0])
timestep = Constant(0.01)

noslip = Constant((0.,0.,0.))
force = Expression(('0.','0.','7*x[2]*x[2]*x[2]*x[2]*x[2]*x[2]/scale'),degree=7,scale=scale)


In [6]:
def solve_unsteady_navier_stokes(W, nu, bcs,forces, T, dt, theta):
    """Solver unsteady Navier-Stokes and write results
    to file"""

    # Current and old solution
    w = Function(W)
    u, p = split(w)

    w_old = Function(W)
    u_old, p_old = split(w_old)

    # Define variational forms
    v, q = TestFunctions(W)
    F = ( Constant(1/dt)*dot(u - u_old, v)
          + Constant(theta)*nu*inner(grad(u), grad(v))
          + Constant(theta)*dot(dot(grad(u), u), v)
          + Constant(1-theta)*nu*inner(grad(u), grad(v))
          + Constant(1-theta)*dot(dot(grad(u_old), u_old), v)
          - p*div(v)
          - q*div(u) - dot(forces,v)
        )*dx
    J = derivative(F, w)

    # Create solver
    problem = NonlinearVariationalProblem(F, w, bcs, J)
    solver = NonlinearVariationalSolver(problem)
    solver.parameters['newton_solver']['linear_solver'] = 'mumps'


    # Perform time-stepping
    t = 0
    while t < T:
        w_old.vector()[:] = w.vector()
        solver.solve()
        t += dt
        print("t = {:.3f}".format(t))
        print("Norm velocity {}".format(norm(w.sub(0))))
        print("Norm pressure {}".format(norm(w.sub(1))))

In [7]:
theta = 1/2
dt = float(timestep)
T = dt*nbitt
solve_unsteady_navier_stokes(W, mu, dbcs,force, T, dt, theta)

Calling FFC just-in-time (JIT) compiler, this may take some time.
           Consider using the option 'quadrature_degree' to reduce the number of points
Calling FFC just-in-time (JIT) compiler, this may take some time.
           Consider using the option 'quadrature_degree' to reduce the number of points
t = 0.010
Calling FFC just-in-time (JIT) compiler, this may take some time.
           Consider using the option 'quadrature_degree' to reduce the number of points
Norm velocity 2.81399316802329e-07
Calling FFC just-in-time (JIT) compiler, this may take some time.
Norm pressure 0.54406009868393
t = 0.020
Norm velocity 2.925151167293212e-07
Norm pressure 0.5440714860678756
t = 0.030
Norm velocity 2.931494687067401e-07
Norm pressure 0.544066234407515
t = 0.040
Norm velocity 2.9320392603694435e-07
Norm pressure 0.5440152428343189
t = 0.050
Norm velocity 2.93211010590303e-07
Norm pressure 0.5440950079881601
t = 0.060
Norm velocity 2.9321238900783056e-07
Norm pressure 0.5440915051207831
t